La práctica ha sido realizada por el grupo L5, que esta compuesto por:

  * Borja Arán Tejada - 100363921
  * Adrián Vázquez Pérez - 100363906
  * Álvaro Iglesias García - 100363759
  * Antonio Rodríguez Arias - 100363883

En esta práctica añadiremos la mejora de: "Reconocimiento de Entidades" propuesto en el enunciado de la práctica 2.3. Para alcanzar el objetivo,  utilizaremos todo el material de la práctica 2.2, exceptuando las métricas, y comparando los diferentes resultados de aquellas consultas que traten acerca de entidades persona. 

Conectamos con Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importamos todas las librerías que vamos a necesitar

In [ ]:

import urllib
from bs4 import BeautifulSoup
import pickle
import re
import string
import os
import xml.etree.ElementTree

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tag import pos_tag
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Definimos las funciones de limpieza que hemos utilizado tanto en la práctica 2.1 como en la práctica 2.2, pero añadimos un método más: _limpiar_persona()_

Necesitamos este método ya que para poder mantener la integridad y coherencia de los nombres proporcionados, solo podemos ejecutar dos funciones definidas en el método _limpieza()_ : _get_text()_ y _remove()_. Actuamos de este modo, porque los nombres propios tienen que estar en su formato original (Ejemplo: John), preservando la primera letra mayúscula, para así, poder identificar de forma inequívoca una entidad persona dentro de un documento. Siguiendo con ese pensamiento, nos vemos obligados a que para poder identificar con éxito dichos documentos, no podemos convertir a minúsculas todo el texto, ni debemos lematizarlo, ya que nos encontraríamos con coincidencias que perjudicarían la precisión de nuestro motor. De tal manera que, el método *limpiar_persona()* 
prepara los documentos para poder evaluar si hay nombres en su contenido.

In [ ]:
def limpieza(corpus):
  corpus = get_text(corpus)
  corpus = remove(corpus)
  corpus = lower_case(corpus)
  corpus = delete_stopwords(corpus)
  corpus = lemmatize_texts(corpus)
  return corpus

def limpiar_persona(corpus):
  corpus = get_text(corpus)
  corpus = remove(corpus)
  return corpus

def get_text(corpus):
  soup = BeautifulSoup(corpus, "html.parser")
  corpus =soup.get_text()
  return corpus

def remove(corpus):
  corpus = corpus.replace('\n', ' ')
  corpus = corpus.replace('\t', ' ')
  corpus = corpus.replace('-', ' ')
  corpus = re.sub('<.*?>', " ", corpus)
  corpus = re.sub('{.*?}', " ", corpus)
  corpus = re.sub(r"\([^()]*\)", " ", corpus)
  corpus = corpus.translate(str.maketrans("","", string.punctuation))
  corpus = re.sub(' +', ' ',corpus)
  return corpus

def lower_case(corpus):
  return corpus.lower()

def delete_stopwords(body):
  clean = ""
  stop_words = set(nltk.corpus.stopwords.words('english'))
  word_tokens = nltk.tokenize.word_tokenize(body)
  for j in word_tokens:
    if not j in stop_words:
      clean = clean + " " +str(j)
  return clean

def lemmatize_texts(corpus):
  clean = ""
  lemm = WordNetLemmatizer()
  tokens = pos_tag(word_tokenize(corpus))
  for word, tag in tokens:
    if pos_tagger(tag) is None:
      clean = clean + " " + str(word)
    else:
      clean = clean + " " +lemm.lemmatize(word, pos_tagger(tag))
  return clean

def pos_tagger(nltk_tag):
  if nltk_tag.startswith('J'):
      return wordnet.ADJ
  elif nltk_tag.startswith('V'):
      return wordnet.VERB
  elif nltk_tag.startswith('N'):
      return wordnet.NOUN
  elif nltk_tag.startswith('R'):
      return wordnet.ADV
  else:          
      return None

Recuperemos la lista de nombres masculinos y femeninos y las unimos en una misma lista, esto lo hacemos para luego tener un string con todos los nombres (separados por un espacio) que utilizaremos para diferenciar los documentos y las queries que hablen de personas con las que no. En este método pasamos por parámetros las dos rutas que llevan a los ficheros de los nombres masculinos y femeninos. Siempre mantenemos la integridad de todos los nombres, sin convertir a minúsculas ninguno de ellos. A modo de conslusión, el método *get_names()* deja en una lista los nombres preparados para ser comparados con el conteido de un documento.

In [ ]:
def get_names(path_docs, path_docs_2):
  names = ""
  with open(path_docs, "r") as file_1:
    for word in file_1:
      word = word.replace("\n", "")
      names = names + " " + word
  with open(path_docs_2, "r") as file_2:
    for word in file_2:
      word = word.replace("\n", "")
      names = names + " " + word
  return names

Ahora creamos dos métodos que devuelva un dict que siga este formato:
{"Index": index, "Persona": X}, de tal manera que siempre que se detecte una entidad persona en un documento html o que se pregunte por una persona en una consulta, estos métodos marcarán respectivamente el documento o la consulta.
En el caso de:
  - Documento html: se dirá que habla de una entidad persona cuando se encuentre en su contenido una palabra que coincida con uno de los nombres proporcionados en los dos ficheros, tanto para nombres masculinos como femeninos.
  - Consulta: si se encuentra cualquiera de estas palabras: "Who", "Whom" o "Whos" ( aún estando en minúsculas) o bien hay una coincidencia con alguno de los nombres propios proporcionados, se asume que dicha consulta esta preguntando por una entidad persona.


De tal manera que siempre tendremos la lista de los documentos y queries original y otra lista de dictionaries que consisten en:
 - Index: Nombre del documento/Index de la query ( Ejemplo: "2010-00-50.html" en el caso de un documento o "3" en el caso de una query)
 - Persona: Con el valor 1 indicamos que habla de personas y con el valor de 0, no se ha detectado ninguna entidad persona.

Este dict es clave para el desarrollo de toda la práctica ya que, consultaremos esta estructura de datos a la hora de ordenar todos los documentos para sumar la constante (k=1), con el fin de que a los documentos en los cuales se detecte una entidad persona para una query que pregunte por una persona, pueda sumársele dicha constante.

Por resumir,  estos métodos se encargan de detectar si en un documento hay un nombre, y por lo tanto, se detecta una entidad persona, o si en una query se pregunta por una entidad persona.



In [ ]:
def detect_persona_html(doc, text, names):
  for element in names.split():
    for word in text.split():
      if element == word:
        return {"Index": doc, "Persona": 1 }
  return {"Index": doc,  "Persona": 0 }

def detect_persona_query(index, query, names):
  for elem in names.split():
    for element in query.split():
      if elem == element or "who" == element or "whos"== element or "whom"== element:
        return {"Index": index,  "Persona": 1 }
  return {"Index": index,  "Persona": 0 }

El primer método es el que utilizamos para importar los documentos y limpiarlos, es el mismo que utilizamos en la práctica 2.2, ya que los documentos html a los que tenemos que acceder son los mismos que los proporcionados en la práctica anterior.

El segundo método es una versión modificada del primero, es el encargado de llamar al método _detect_persona_html()_. Es necesario hacer esta modificación, ya que para buscar si existen nombres de persona en el contenido  de los ficheros html nos vemos obligados a hacer solo dos funciones de limpieza: _get_text()_ y _remove()_

Esta decisión ha sido tomada porque si seguimos el método de _limpieza()_ seguido en la práctica 2.2, no podemos hacer ni _lower_case_() ni lemmatizar,porque entonces no se respetarían las mayúsculas que un nombre propio debe tener para ser identificado, lo cual daría mucho ruido a nuestros resultados.

In [ ]:

def get_docs(path_docs, docs, html, corpus):
  i = 0
  aux = []
  for dirpath, dirnames, filenames in os.walk(path_docs):
    for filename in filenames:
      docs.append(filename)
      html.append(open (os.path.join(dirpath, filename), 'r'))
      corpus.append(limpieza(html[i]))
      i = i+1

def get_docs_persona(path_docs, html_persona, names, docs_persona):
    i = 0
    aux = []
    for dirpath, dirnames, filenames in os.walk(path_docs):
      for filename in filenames:
        html_persona.append(open (os.path.join(dirpath, filename), 'r'))
        docs_persona.append(detect_persona_html(filename, limpiar_persona(html_persona[i]), names))
        i=i+1

Este método es el que utilizamos para importar el documento _topics.html_ y limpiarlo, es el mismo que utilizamos en la práctica 2.2, ya que las consultas a tener en cuenta son las mismas que en la práctica anterior, salvo con una pequeña variación, la cual observamos en la primera línea dentro del bucle for.

Esta línea es la encargada de detectar las querys que preguntan por una persona. Este método lo veremos explicado más detalladamente en su respectiva definición, pero básicamente si se encuentra la palabra "Who", "Whom" o "Whos", se asumirá que pregunta por una entidad persona.

In [ ]:
def get_queries(path_queries, queries_recuperadas, names):
  queries = []
  ubicacion = xml.etree.ElementTree.parse(path_queries).getroot()
  i = 0
  for query in ubicacion.findall('topic'):
    queries_recuperadas.append(detect_persona_query(i, lower_case(query.find('title').text), names))
    queries.append(limpieza(query.find('title').text))
    i = i+1
  return queries

Este método calcula la función de similitud del coseno tfidf, con el fin de otorgar un valor numérico que de sentido a nuestro motor de búsqueda. Para ello, necesitamos todas las consultas y el corpus, ambos necesarios para poder vectorizar y calcular la función con éxito.

Por último, transponemos la matriz para que nos sea más cómodo recorrerla y operar con ella, convirtiendo las filas en los documentos y las columnas en las consultas.

In [ ]:
def do_matriz(corpus, queries):
  tfidf_vectorizer = TfidfVectorizer()
  matriz_tfidf = tfidf_vectorizer.fit_transform(corpus)
  queries_tfidf =tfidf_vectorizer.transform(queries)
  tfidf_matriz = cosine_similarity(queries_tfidf, matriz_tfidf)
  matriz = np.transpose(tfidf_matriz)
  return matriz


Aquí definimos dos métodos que serán el resultado de la práctica:
  - El primero ("relevantes_ordenados"): ordena los documentos sin aplicar la mejora de la detección de entidades de tipo persona.
  - El segundo ("relevantes_persona"): consiste en una variación del primer método, donde sumamos una constante (k=1) a aquellos documentos que hablen de entidades personas para cada consulta que hable de entidades persona. En este método, también observamos una variación adicional, el aumento de la variable umbral, esta decisión será explicada posteriormente al hacer un análisis de los resultados obtenidos en este motor.

Ambos métodos devuelven una lista con los documentos ordenados de mayor a menor según la relevancia obtenida por nuestro motor.

In [ ]:
def relevantes_ordenados(index, umbral, matriz, docs, queries):
  dic = []
  ordenados = []
  for i in range(len(matriz)):
    ordenados.append(matriz[i][(index)])
  ordenados.sort(reverse=True)
  for i in range(len(ordenados)):
    if ordenados[i] >= umbral:
      for j in range(len(matriz)):
        if matriz[j][index]==ordenados[i]:
          dic.append(docs[j])
  return dic

def relevantes_persona(index, umbral, matriz, docs, queries, docs_persona):
  umbral = umbral + 1.05
  dic = []
  ordenados = []
  for i in range(len(matriz)):
    if (docs_persona[i]["Persona"]==0):
      ordenados.append(matriz[i][(index)])
    if (docs_persona[i]["Persona"]==1):
      ordenados.append(matriz[i][(index)]+1)
  ordenados.sort(reverse=True)
  for i in range(len(ordenados)):
    if ordenados[i] >= umbral:
      for j in range(len(matriz)):
        if matriz[j][index]+1==ordenados[i] and docs_persona[j]["Persona"]==1:
          dic.append(docs[j])
        elif (matriz[j][index])==ordenados[i] and docs_persona[j]["Persona"]==0:
          dic.append(docs[j])
  return dic

Ejecución final de todo el código explicado anteriormente:

Será necesario adaptar las rutas de los archivos en Drive para que pueda ejecurtarse con éxito.

Seguimos el siguiente orden:

1. Obtenemos los nombres de las personas y los dejamos en una lista.
2. Obtenemos el contenido de los documentos preparado para vectorizar.
3. Obtenemos aquellos documentos que hablen de personas.
4. Obtenemos las queries listas para vectorizar y en el mismo método detectamos las queries que preguntan por una persona.
5. Obtenemos la matriz TFIDF.
6. Obtenemos los recuperados ordenados para toda consulta, y si alguna query se detecta que pregunta por una persona, se llamará al método que también muestra la lista teniendo en cuenta la mejora de detección de entidades persona.

In [ ]:
path_docs = "/content/drive/MyDrive/RAI/Practica2/2.3/documents.biased"
path_queries = "/content/drive/MyDrive/RAI/Practica2/2.3/topics.xml"
path_names = "/content/drive/MyDrive/RAI/Practica2/2.3/female.txt"
path_names_2 = "/content/drive/MyDrive/RAI/Practica2/2.3/male_v2.txt"

docs = []
html = []
html_persona = []
corpus = []
docs_persona = []
queries_persona = []
recuperados = []
recuperados_persona = []


names = get_names(path_names, path_names_2)
get_docs(path_docs, docs, html, corpus)
get_docs_persona(path_docs, html_persona, names, docs_persona)
queries = get_queries(path_queries, queries_persona, names)
matriz = do_matriz(corpus, queries)

Estas pocas líneas de código nos darán los resultados obtenidos para el ejercicio propuesto y procedemos a explicar el formato en el que visualizaremos los datos.

Vamos recorriendo las consultas por orden y si se ha detectado que una query pregunta por una entidad persona, se muestran los documentos html relevantes a esta query, teniendo en cuenta la mejora de las entidad persona. Después de esta condición, siempre se enseña el orden de los documentos relevantes según nuestro motor, consiguiendo así mantener la integridad y mostrar siempre resultados para queries que no pregunten por personas, y además, poder comparar una query que pregunte por personas con el orden de los documentos si no se tuviera en cuenta la mejora de detección de entidades del tipo persona.

In [ ]:
umbral = 0.09
i=0
for query in queries:
  print("Consulta "+ str(queries.index(query)+1))
  if queries_persona[int(queries.index(query))]["Persona"]==1:
      print("SE HA DETECTADO UNA ENTIDAD PERSONA EN ESTA QUERY")
      print("DOCUMENTOS RECUPERADOS POR ORDEN DE RELEVANCIA (ENTIDAD PERSONA ON)")
      print(relevantes_persona(int(queries.index(query)), umbral, matriz, docs, queries, docs_persona))
  print("RECUPERADOS POR ORDEN DE RELEVANCIA (ENTIDAD PERSONA OFF)")
  print(relevantes_ordenados(int(queries.index(query)), umbral, matriz, docs, queries))
  print()
  i = i+1

Consulta 1
RECUPERADOS POR ORDEN DE RELEVANCIA (ENTIDAD PERSONA OFF)
['2010-69-054.html', '2010-40-061.html', '2010-12-045.html', '2010-58-011.html', '2010-05-084.html', '2010-37-081.html', '2010-15-015.html', '2010-48-049.html', '2010-06-051.html', '2010-33-011.html', '2010-90-009.html', '2010-88-024.html', '2010-75-054.html', '2010-16-057.html', '2010-21-023.html', '2010-74-080.html', '2010-17-054.html', '2010-11-018.html', '2010-49-061.html', '2010-59-076.html', '2010-17-083.html', '2010-77-012.html', '2010-00-094.html', '2010-33-083.html', '2010-08-049.html', '2010-43-060.html', '2010-84-061.html', '2010-11-029.html', '2010-56-062.html', '2010-19-056.html', '2010-05-007.html', '2010-41-070.html', '2010-07-104.html', '2010-51-039.html', '2010-56-041.html', '2010-52-067.html', '2010-40-008.html', '2010-08-008.html', '2010-29-022.html', '2010-82-105.html', '2010-37-008.html', '2010-64-010.html', '2010-95-002.html', '2010-59-010.html', '2010-31-054.html', '2010-99-036.html', '2010-33-0

En este pequeño fragmento de código calculamos cuantos documentos se han detectado que hablen de una entidad persona (1824 documentos) y procedemos a analizar los resutados obtenidos en el output anterior.

In [ ]:
i = 0
j = 0
for x in docs_persona:
  if docs_persona[i]["Persona"]==1:
    j = j+1
  i = i+1
print(j)

1824


Nos encontramos con que apenas ha habido cambios entre las dos listas proporcionadas en las consultas detectadas que pregunten por personas, esto se debe principalmente a que se detecta que en 1824 documentos tienen al menos una entidad persona en su contenido. Teniendo en cuenta que hay algo más de 2000 documentos, más de un 90% de los documentos están marcados como que contienen al menos una entidad persona.

Este es un error que creemos que deriva del concepto POS (Part of Speech), ya que existen nombres de empresas que tienen un nombre propio en su interior que se detectan como _Chandler Realty, Inc_ en el documento _2010-00-50.html_ , lo cual hace que se detecten entidades falsas de personas generando mucho ruido. Hemos pensado que se podría mejorar enormemente estos resultados aplicando POS, ya que podríamos detectar cuando realmente se habla de una persona, con la ayuda del contexo del documento en cuestión.

El hecho de que un 90% de los documentos contienen nombres de personas, provoca que los resultados del motor sean casi inmutables, ya que por probabilidad los documentos que eran los más relevantes para aquellas consultas que pregunten por personas, sin tener en cuenta la mejora implementada, seguirán siendo los más relevantes después de aplicar la mejora de detección de entidades persona.

Esta situación nos obliga a que cuando una consulta pregunte por una entidad persona, como los documentos relevantes solamente serán aquellos que hablen de personas, aumentamos el umbral definido previamente en "1.21". Este valor procedente del umbral, se debe a sumar la constante definida al detectar documentos que contengan personas en su interior (k=1) y "0.21" ya que consideramos que los documentos que sean relevantes deben tener un valor más elevado en la función de similitud calculada. 

Como resultado final, observamos que la lista devuelta, cuando se detecta en una query una entidad persona, el orden es muy similar al aplicar la mejora implementada respecto a no aplicarla, pero en el primer caso, se devuelve una lista más corta ya que el motor es más exigente.